# import and read

In [28]:
import pandas as pd
import os
import sqlalchemy as sql
import numpy as np


In [29]:
# Verbindung zur PostgreSQL-Datenbank herstellen
# Hier musst du die tatsächlichen Verbindungsdaten deiner PostgreSQL-Datenbank eingeben
db_username = 'dein_benutzername'
db_password = 'dein_passwort'
db_host = 'localhost'
db_port = '5432'
db_name = 'zensus'

engine = create_engine(f'postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

# Pfad zur Excel-Datei
file_path = 'C:\\Users\\hp\\OneDrive\\Desktop\\DBU\\SP3-1.Dateninfrastrukturen\\Pruefung\\Zensus\\input\\Zensus.xlsx'
# Lese die Excel-Datei ein und speichere die Blätter als einzelne DataFrames
sheets = pd.read_excel(file_path, sheet_name=None)
csv_sheets = {sheet_name: data for sheet_name, data in sheets.items() if sheet_name.startswith('CSV')}

# check
print("Dateninfrastruktur aufgesetzt und Excel-Daten geladen.")

Dateninfrastruktur aufgesetzt und Excel-Daten geladen.


In [30]:
# Den Pfad kontrollieren
db_path = os.path.abspath('zensus.db')
print(f"Die SQLite-Datenbank wurde hier erstellt: {db_path}")

Die SQLite-Datenbank wurde hier erstellt: c:\Users\hp\OneDrive\Desktop\DBU\SP3-1.Dateninfrastrukturen\Pruefung\Zensus\notebooks\zensus.db


In [31]:
sheet_names = csv_sheets.keys()
print(sheet_names)

dict_keys(['CSV-Klassenstufe', 'CSV-Schulform', 'CSV-Hoechster_Schulabschluss', 'CSV-Hoechster_berufl_Abschluss', 'CSV-Erwerbsstatus', 'CSV-ET_Alter', 'CSV-ET_Hoechst_berufl_Abschl', 'CSV-ET_Stellung_im_Beruf', 'CSV-ET_Beruf_Hauptgr_ISCO08', 'CSV-ET_Wirtschaftszweig'])


In [32]:
# Speichern der DataFrames in separaten Variablen mit angepassten Namen
created_dfs = []
for sheet_name, data in csv_sheets.items():
    new_sheet_name = sheet_name.replace('-', '_')
    globals()[new_sheet_name] = data
    created_dfs.append(new_sheet_name)

# Anzeigen der neuen Namen der erstellten DataFrames
print("Erstellte DataFrames:")
for df_name in created_dfs:
    print(df_name)

Erstellte DataFrames:
CSV_Klassenstufe
CSV_Schulform
CSV_Hoechster_Schulabschluss
CSV_Hoechster_berufl_Abschluss
CSV_Erwerbsstatus
CSV_ET_Alter
CSV_ET_Hoechst_berufl_Abschl
CSV_ET_Stellung_im_Beruf
CSV_ET_Beruf_Hauptgr_ISCO08
CSV_ET_Wirtschaftszweig


In [ ]:
# Überprüfung auf fehlende Werte in allen DataFrames
for i, df_name in enumerate(created_dfs):
    df = globals()[df_name]
    print(f"Missing values in DataFrame {i + 1} ({df_name}):")
    print(df.isnull().sum())
   

# EDA

In [42]:
CSV_Klassenstufe.head(2)

,Berichtszeitpunkt,_RS,Name,Regionalebene,SCHUELER_KLSS_STP,SCHUELER_KLSS_STP__M,SCHUELER_KLSS_STP__W,SCHUELER_KLSS_STP__1,SCHUELER_KLSS_STP__1_M,SCHUELER_KLSS_STP__1_W,SCHUELER_KLSS_STP__2,SCHUELER_KLSS_STP__2_M,SCHUELER_KLSS_STP__2_W,SCHUELER_KLSS_STP__3,SCHUELER_KLSS_STP__3_M,SCHUELER_KLSS_STP__3_W
0,20220515,0,Deutschland,Bund,8903780,4536040,4367740,3048570,1563180,1485400,4490850,2324270,2166580,1364360,648590,715760
1,20220515,1,Schleswig-Holstein,Land,320060,162880,157180,109240,54930,54310,161100,83860,77240,49720,24090,25630


In [44]:
CSV_Schulform.head(2)

,Berichtszeitpunkt,_RS,Name,Regionalebene,SCHUELER_SCH_STP,SCHUELER_SCH_STP__M,SCHUELER_SCH_STP__W,SCHUELER_SCH_STP__1,SCHUELER_SCH_STP__1_M,SCHUELER_SCH_STP__1_W,SCHUELER_SCH_STP__2,SCHUELER_SCH_STP__2_M,SCHUELER_SCH_STP__2_W,SCHUELER_SCH_STP__3,SCHUELER_SCH_STP__3_M,SCHUELER_SCH_STP__3_W,SCHUELER_SCH_STP__4,SCHUELER_SCH_STP__4_M,SCHUELER_SCH_STP__4_W,SCHUELER_SCH_STP__5,SCHUELER_SCH_STP__5_M,SCHUELER_SCH_STP__5_W,SCHUELER_SCH_STP__6,SCHUELER_SCH_STP__6_M,SCHUELER_SCH_STP__6_W,SCHUELER_SCH_STP__7,SCHUELER_SCH_STP__7_M,SCHUELER_SCH_STP__7_W
0,20220515,0,Deutschland,Bund,8903780,4536040,4367740,3033030,1544890,1488150,231290,147550,83740,547660,281880,265780,406900,234310,172590,1251860,654280,597590,838400,433130,405270,2594630,1240010,1354630
1,20220515,1,Schleswig-Holstein,Land,320060,162880,157180,105520,52830,52690,5680,3870,1800,32230,16820,15410,6360,3710,2660,16560,8850,7710,60880,31750,29130,92830,45050,47780


# transform

In [34]:
df=pd.DataFrame(CSV_Klassenstufe)

In [35]:
# Optionen einstellen
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

# DataFrame anzeigen
print(df.head())

   Berichtszeitpunkt          _RS                    Name                          Regionalebene SCHUELER_KLSS_STP SCHUELER_KLSS_STP__M SCHUELER_KLSS_STP__W SCHUELER_KLSS_STP__1 SCHUELER_KLSS_STP__1_M SCHUELER_KLSS_STP__1_W SCHUELER_KLSS_STP__2 SCHUELER_KLSS_STP__2_M SCHUELER_KLSS_STP__2_W SCHUELER_KLSS_STP__3 SCHUELER_KLSS_STP__3_M SCHUELER_KLSS_STP__3_W
0           20220515            0             Deutschland                                   Bund           8903780              4536040              4367740              3048570                1563180                1485400              4490850                2324270                2166580              1364360                 648590                 715760
1           20220515            1      Schleswig-Holstein                                   Land            320060               162880               157180               109240                  54930                  54310               161100                  83860                 

In [36]:
df['SCHUELER_KLSS_STP'] = pd.to_numeric(df['SCHUELER_KLSS_STP'], errors='coerce')

In [37]:
df_cleaned = df.dropna(subset=['SCHUELER_KLSS_STP'])

In [38]:
df_cleaned.groupby('Regionalebene')['SCHUELER_KLSS_STP'].sum()

Regionalebene
Bund                                     8903780.0
Gemeinde                                 6631640.0
Gemeindeverband                          1310750.0
Land                                     8903780.0
Regierungsbezirk                         7013880.0
Stadtkreis/kreisfreie Stadt/Landkreis    8903770.0
Name: SCHUELER_KLSS_STP, dtype: float64

In [39]:
print(df['Regionalebene'].unique())

['Bund' 'Land' 'Stadtkreis/kreisfreie Stadt/Landkreis' 'Gemeinde'
 'Gemeindeverband' 'Regierungsbezirk']


In [40]:
# Verbindung zur Datenbank schließen
engine.dispose()